In [14]:
import pandas as pd
import bs4 as bs
import requests
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [182]:
df = pd.read_csv('/content/drive/MyDrive/data_490/processed_data/pandas_30_day')
players = df.player_name.unique()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (89,95,96,97,99,105,106,107) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [183]:
spotrac = pd.read_csv('/content/drive/MyDrive/data_490/injury_data/injury_history')

In [184]:
spotrac = spotrac.rename(columns={'Covid 19':'COVID'})
injury_types = spotrac.columns[2:].values

In [186]:
injury_df = pd.DataFrame(columns=['player_name', 'date', 'notes'])

In [197]:
for player in players[1857:]:
  url = "https://www.prosportstransactions.com/baseball/Search/SearchResults.php?Player=Clayton+Kershaw&Team=&BeginDate=&EndDate=&DLChkBx=yes&submit=Search"
  name_split = player.split()
  name = "".join([f'{name}+' for name in name_split])[:-1]
  url = url.replace('Clayton+Kershaw', name)
  data = requests.get(url).text
  soup = bs.BeautifulSoup(data, 'html.parser')
  table = soup.find('table', class_='datatable center')
  #If the player has no injury history continue
  if table is None:
    continue
  for i, row in enumerate(table.find_all('tr')):
    columns = row.find_all('td')
    #Checks for first row is the heading row and skips heading row
    if columns[0].text.strip() == 'Date':
      continue
    #Only care about when player is placed on injured list not when activated
    if columns[-1].text.strip().split(' ')[0] == 'placed':
      #adds new row to df
      new = {'player_name': player, 'date': columns[0].text.strip(), 'notes': columns[-1].text.strip()}
      injury_df = injury_df.append(new, ignore_index=True)

In [224]:
for i, injury in enumerate(injury_types):
  injury_types[i] = injury.lower()

In [200]:
injury_df.notes = injury_df.apply(lambda row: row.notes.lower(),axis=1)

In [227]:
def get_injury_type(row):
  for injury in injury_types:
    if injury in row.notes:
      return injury
  return 'undisclosed'

In [228]:
injury_df['injury_type'] = injury_df.apply(get_injury_type, axis=1)

In [229]:
#remove rows where the player is put on the covid list
injury_df = injury_df[injury_df.injury_type != 'covid']

In [230]:
injury_df

,player_name,date,notes,injury_type
0,A.J. Burnett,2001-03-31,placed on 15-day dl,undisclosed
1,A.J. Burnett,2002-08-19,placed on 15-day dl with bone bruise on right ...,elbow
2,A.J. Burnett,2003-03-29,placed on 15-day dl with right elbow inflammation,elbow
3,A.J. Burnett,2003-04-28,placed on 15-day dl with right elbow inflammation,elbow
4,A.J. Burnett,2004-03-26,placed on 15-day dl with right elbow injury,elbow
...,...,...,...,...
4501,Willians Astudillo,2019-06-27,placed on 10-day il with strained left oblique,oblique
4503,Yermín Mercedes,2022-04-07,placed on 10-day il with fractured left hamate...,undisclosed
4504,Todd Frazier,2018-05-08,placed on 10-day dl with strained left hamstring,hamstring
4505,Todd Frazier,2018-07-09,placed on 10-day dl with strained left rib cage,undisclosed


In [258]:
count = injury_df.groupby('player_name').size()

In [263]:
dates = [f'date_{i}' for i in range(1, count.max()+1)]
types = [f'injury_type_{i}' for i in range(1, count.max()+1)]

In [282]:
new = injury_df
new['new'] = injury_df.groupby('player_name').cumcount()+1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [283]:
injury_df = new.set_index(['player_name', 'new']).unstack('new')

In [284]:
injury_df.columns = injury_df.columns.map('{0[0]}{0[1]}'.format)

In [288]:
injury_df = injury_df.reset_index()
injury_df

,player_name,date1,date2,date3,date4,date5,date6,date7,date8,date9,...,injury_type4,injury_type5,injury_type6,injury_type7,injury_type8,injury_type9,injury_type10,injury_type11,injury_type12,injury_type13
0,A.J. Burnett,2001-03-31,2002-08-19,2003-03-29,2003-04-28,2004-03-26,2006-03-31,2006-04-22,2007-06-19,2007-07-01,...,elbow,elbow,elbow,elbow,shoulder,shoulder,undisclosed,calf,NaN,NaN
1,A.J. Cole,2018-06-21,2019-08-10,2021-06-02,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A.J. Griffin,2012-08-05,2014-03-29,2015-02-27,2015-09-01,2016-05-08,2017-04-21,2017-05-27,NaN,NaN,...,shoulder,shoulder,ankle,undisclosed,NaN,NaN,NaN,NaN,NaN,NaN
3,A.J. Minter,2019-03-28,2019-09-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,A.J. Morris,2016-06-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1297,Zack Britton,2011-08-05,2012-03-26,2017-04-16,2017-05-06,2018-02-16,2020-08-20,2021-03-31,2021-06-26,2021-08-23,...,arm,achilles,hamstring,elbow,hamstring,elbow,elbow,NaN,NaN,NaN
1298,Zack Godley,2020-09-10,2021-04-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1299,Zack Greinke,2006-03-31,2011-03-26,2013-04-12,2016-07-03,2021-09-24,NaN,NaN,NaN,NaN,...,oblique,neck,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1300,Zack Littell,2020-08-01,2020-08-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [314]:
#Drops notes columns, only keeps dates columns and injury type columns
injury_df = pd.concat([injury_df.iloc[:,:14], injury_df.iloc[:,-13:]], axis=1)

In [316]:
injury_df.to_csv('/content/drive/MyDrive/data_490/injury_data/scraped_injury.csv',index=False)